# Prise en main des Transformer 



In [ ]:
# import standard + 
# 
import numpy as np

import torch.nn.functional as F
import torch
import torch.nn as nn

import os
import time
import logging
import re
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = "cpu"
print(device)

torch.manual_seed(42)

In [ ]:
# Paramètres du modèle
input_dim = 3          # Dimension des vecteurs d'entrée
seq_length = 10        # Longueur de la séquence
num_heads = 1          # Nombre de têtes d'attention
hidden_dim = 6        # Dimension de la couche intermédiaire dans le feedforward
num_layers = 1         # Nombre de couches du TransformerEncoder
batch_size = 8         # Taille du batch

# Création d'une couche TransformerEncoderLayer
encoder_layer = nn.TransformerEncoderLayer(
    d_model=input_dim, 
    nhead=num_heads, 
    dim_feedforward=hidden_dim,
    activation='relu'
)

# Création d'un TransformerEncoder avec plusieurs couches
transformer_encoder = nn.TransformerEncoder(
    encoder_layer, 
    num_layers=num_layers
)

# Génération de données aléatoires
data = torch.randn(seq_length, batch_size, input_dim)  # Shape: (seq_length, batch_size, input_dim)

# Passer les données dans l'encodeur
encoded_output = transformer_encoder(data)

print("input shape:", data.shape)
print("Encoded output shape:", encoded_output.shape)
# print("Encoded output:", encoded_output)

### Analyse de l'évolution des embeddings

Entrée vs Sortie: comme les embeddgins sont en 2D, on peut directement les tracer. Evidemment, on ne trace que pour un élément du batch (sinon, ça n'a pas de sens)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i = 1 # premier element
input  = data[:,i,:].squeeze().detach().numpy()
output = encoded_output[:,i,:].squeeze().detach().numpy()


fig = plt.figure()
ax = fig.add_subplot(projection='3d')

for ei,eo in zip(input,output):
    # print(ei,eo)
    ax.scatter([ei[0], eo[0]], [ei[1], eo[1]], [ei[2], eo[2]])
    ax.plot([ei[0], eo[0]], [ei[1], eo[1]], [ei[2], eo[2]])

ax.legend(np.arange(len(input)*2)//2)

In [ ]:
input

### Analyse de la matrice d'attention

In [ ]:
in_proj_weight = encoder_layer.self_attn.in_proj_weight
in_proj_bias = encoder_layer.self_attn.in_proj_bias

in_proj_weight.size()
# data.size()

In [ ]:
import math
import torch.nn.functional as F

d = in_proj_weight.size(1)
KQV = data[:,i,:] @ in_proj_weight.T + in_proj_bias # tjs pour la seq i
# la suite ne marche que pour une tête
K = KQV[:,:d]
Q = KQV[:,d:2*d]

attn = F.softmax(K@Q.T/math.sqrt(d),dim=1)

plt.figure()
plt.imshow(attn.detach().numpy())
plt.colorbar()

# vérification des sommes:
print("col : ", attn.sum(0))
print("line : ", attn.sum(1))

# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###